In [1]:
import  torch

import sys
import pytorch_lightning as pl

# setting path
sys.path.append('~/pubmedQA_291')
from dataset import MedMCQA_Datamodule
from model import MedMCQAModel
from show_attention import generate
config = {
    "contrastive": False,
    "has_additional_prompt":False,
    "pretrained_model":"microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext",
    "num_workers": 8,
    "bs":32,
    "lr":2e-5,
    "attention_dropout" : 0.1,
    "hidden_dropout" : 0.1,
    "weight_decay": 1e-3,
    "model_style": "classification",
    "output_attentions": True
}
dm = MedMCQA_Datamodule(batch_size=64, parser_args=config)
model = MedMCQAModel(parser_args=config, **config)
path_finetuned_pubmed_bert = '/root/pubmedQA_291/exps/prevent_overfitting/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext_classification/pubmed_bert/checkpoints/epoch=2-step=4287.ckpt'


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
trainer = pl.Trainer(
                        max_epochs=1,
                        accelerator="gpu",
                        
                    )

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:


trainer.validate(model, dm, ckpt_path=path_finetuned_pubmed_bert)


Restoring states from the checkpoint path at /root/pubmedQA_291/exps/prevent_overfitting/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext_classification/pubmed_bert/checkpoints/epoch=2-step=4287.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at /root/pubmedQA_291/exps/prevent_overfitting/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext_classification/pubmed_bert/checkpoints/epoch=2-step=4287.ckpt


Validation DataLoader 0: 100%|██████████| 66/66 [00:14<00:00,  4.61it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        val/ce_loss        │    1.4994843006134033     │
│       val/mean_acc        │    0.42690834403038025    │
└───────────────────────────┴───────────────────────────┘

[{'val/ce_loss': 1.4994843006134033, 'val/mean_acc': 0.42690834403038025}]

In [6]:
config['has_additional_prompt'] = True
dm_hints = MedMCQA_Datamodule(batch_size=64, parser_args=config)
model_hints = MedMCQAModel(parser_args=config, **config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
trainer.validate(model_hints, dm_hints, ckpt_path=path_finetuned_pubmed_bert)


Restoring states from the checkpoint path at /root/pubmedQA_291/exps/prevent_overfitting/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext_classification/pubmed_bert/checkpoints/epoch=2-step=4287.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at /root/pubmedQA_291/exps/prevent_overfitting/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext_classification/pubmed_bert/checkpoints/epoch=2-step=4287.ckpt


Validation DataLoader 0: 100%|██████████| 66/66 [00:17<00:00,  3.78it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        val/ce_loss        │    1.5857653617858887     │
│       val/mean_acc        │    0.3806612491607666     │
└───────────────────────────┴───────────────────────────┘

[{'val/ce_loss': 1.5857653617858887, 'val/mean_acc': 0.3806612491607666}]

In [10]:
path_contrastive_model = '/root/pubmedQA_291/exps/contrastive/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext_classification/pubmed_bert_contrastive_finetuning_small_lr/checkpoints/epoch=0-step=1429.ckpt'
config['contrastive'] = True
config['contrastive_coeff']=0.1 
config['has_additional_prompt'] = False

In [11]:
# contrastive model, no hints


dm_contrastive = MedMCQA_Datamodule(batch_size=64, parser_args=config)
model_contrastive = MedMCQAModel(parser_args=config, **config)
trainer.validate(model_contrastive, dm_contrastive, ckpt_path=path_contrastive_model)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Restoring states from the checkpoint path at /root/pubmedQA_291/exps/contrastive/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext_classification/pubmed_bert_contrastive_finetuning_small_lr/checkpoints/epoch=0-step=1429.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at /root/pubmedQA_291/exps/contrastive/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext_classification/pubmed_bert_contrastive_finetuning_small_lr/checkpoints/epoch=0-step=1429.ckpt


Validation DataLoader 0: 100%|██████████| 66/66 [00:13<00:00,  4.78it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        val/ce_loss        │    1.8113086223602295     │
│       val/mean_acc        │    0.4288022816181183     │
└───────────────────────────┴───────────────────────────┘

[{'val/ce_loss': 1.8113086223602295, 'val/mean_acc': 0.4288022816181183}]

In [12]:
config['has_additional_prompt'] = True

dm_contrastive_w_hints = MedMCQA_Datamodule(batch_size=64, parser_args=config)
model_contrastive_w_hints = MedMCQAModel(parser_args=config, **config)
trainer.validate(model_contrastive_w_hints, dm_contrastive_w_hints, ckpt_path=path_contrastive_model)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Restoring states from the checkpoint path at /root/pubmedQA_291/exps/contrastive/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext_classification/pubmed_bert_contrastive_finetuning_small_lr/checkpoints/epoch=0-step=1429.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at /root/pubmedQA_291/exps/contrastive/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext_classification/pubmed_bert_contrastive_finetuning_small_lr/checkpoints/epoch=0-step=1429.ckpt


Validation DataLoader 0: 100%|██████████| 66/66 [00:17<00:00,  3.79it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        val/ce_loss        │    1.8931595087051392     │
│       val/mean_acc        │    0.3915822505950928     │
└───────────────────────────┴───────────────────────────┘

[{'val/ce_loss': 1.8931595087051392, 'val/mean_acc': 0.3915822505950928}]

In [2]:
from transformers import BertTokenizer
# dm_hints.setup()
# val_loader = dm_hints.val_dataloader()
path_contrastive_model = '/root/pubmedQA_291/exps/contrastive/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext_classification/pubmed_bert_contrastive/checkpoints/epoch=6-step=10003.ckpt'
config['contrastive'] = True
config['contrastive_coeff']=0.1 
config['has_additional_prompt'] = False


dm_contrastive = MedMCQA_Datamodule(batch_size=1, parser_args=config)
model_contrastive = MedMCQAModel(parser_args=config, **config)

dm_contrastive.setup()
val_loader = dm_contrastive.val_dataloader()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
model= MedMCQAModel.load_from_checkpoint(checkpoint_path=path_contrastive_model, parser_args=config)

tokenizer = BertTokenizer.from_pretrained(config['pretrained_model'])
special_tokens = set(tokenizer.all_special_tokens)
special_tokens

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'[CLS]', '[MASK]', '[PAD]', '[SEP]', '[UNK]'}

In [74]:

for ite, batch in enumerate(val_loader):
    
    if ite<2: continue
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask = input_ids.cuda(), attention_mask.cuda()
    # inputs = tokenizer.encode(input_text, return_tensors='pt')  # Tokenize input text
    outputs = model(input_ids, attention_mask, output_hidden_states=True)  # Run model
    pred =  torch.argmax(outputs.logits, dim=1)
    print(f"labl: {labels}, pred: {pred}")
    attention = outputs.attentions  # Retrieve attention from model outputs
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])  # Convert input ids to token strings
    
    attention_sum = sum(attention[0:6])[:,:,0].sum(dim=(0,1)) # Sum the attention weights across heads in first layers
    # attention_sum = attention[-1][:,:,0].sum(dim=(0,1)) # Sum the attention weights across heads in first layers
    
    # if ite==10:
    #     break
    break

labl: tensor([2]), pred: tensor([2], device='cuda:0')


In [75]:
attention_to_show = torch.clip(attention_sum*30, 0 ,100).tolist()
texts_and_attention_weights = [(tokens[i],attention_to_show[i]) for i  in range(len(tokens)) if tokens[i] not in special_tokens]
texts, attention_weights_for_text = zip(*texts_and_attention_weights)

In [76]:
generate(texts, attention_weights_for_text, "no_hint.tex")

In [60]:
tokens

['[CLS]',
 'subject',
 ':',
 'medicine',
 ',',
 'topic',
 ':',
 'none',
 'question',
 ':',
 'a',
 '29',
 'yrs',
 'old',
 'woman',
 'with',
 'a',
 'pregnancy',
 'of',
 '17',
 'week',
 'has',
 'a',
 '10',
 'years',
 'old',
 'boy',
 'with',
 'down',
 'syndrome',
 '.',
 'she',
 'does',
 'not',
 'want',
 'another',
 'down',
 'syndrome',
 'kid',
 ';',
 'best',
 'advice',
 'to',
 'her',
 'is',
 'a',
 ':',
 'no',
 'test',
 'is',
 'required',
 'now',
 'as',
 'her',
 'age',
 'is',
 'below',
 '35',
 'years',
 'b',
 ':',
 'ultra',
 'sound',
 'at',
 'this',
 'point',
 'of',
 'time',
 'will',
 'definitely',
 'tell',
 'her',
 'that',
 'next',
 'baby',
 'will',
 'be',
 'down',
 'syndromic',
 'or',
 'not',
 'c',
 ':',
 'amniotic',
 'fluid',
 'samples',
 'plus',
 'chromosomal',
 'analysis',
 'will',
 'definitely',
 'tell',
 'her',
 'that',
 'next',
 'baby',
 'will',
 'be',
 'down',
 'syndromic',
 'or',
 'not',
 'd',
 ':',
 'blood',
 'screening',
 'at',
 'this',
 'point',
 'of',
 'time',
 'will',
 'clear

In [23]:
config['has_additional_prompt'] = True
dm_contrastive_hints = MedMCQA_Datamodule(batch_size=1, parser_args=config)
dm_contrastive_hints.setup()
val_loader_hints = dm_contrastive_hints.val_dataloader()


In [77]:

for ite, batch in enumerate(val_loader_hints):
    if ite<2: continue
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask = input_ids.cuda(), attention_mask.cuda()
    # inputs = tokenizer.encode(input_text, return_tensors='pt')  # Tokenize input text
    outputs = model(input_ids, attention_mask, output_hidden_states=True)  # Run model
    print(f"labl: {labels}, pred: {pred}")
    # if labels.item()!=pred:
    #    continue
    attention = outputs.attentions  # Retrieve attention from model outputs
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])  # Convert input ids to token strings
    
    pred =  torch.argmax(outputs.logits, dim=1)
    # bad = False
    # for t in tokens:
    #     if "#" in t:
    #         bad = True
    #         break
    # if bad:
    #     continue
    
    attention_sum = sum(attention)[:,:,0].sum(dim=(0,1)) # Sum the attention weights across heads in first layers
    
    break
 

labl: tensor([2]), pred: tensor([2], device='cuda:0')


In [25]:
ite

42

In [78]:
attention_to_show = torch.clip(attention_sum*30, 0 ,100).tolist()
texts_and_attention_weights = [(tokens[i],attention_to_show[i]) for i  in range(len(tokens)) if tokens[i] not in special_tokens]
texts, attention_weights_for_text = zip(*texts_and_attention_weights)
generate(texts, attention_weights_for_text, "with_hint.tex")

In [27]:
tokens

['[CLS]',
 'subject',
 ':',
 'pediatrics',
 ',',
 'topic',
 ':',
 'none',
 'question',
 ':',
 'given',
 'a',
 'retrospective',
 'study',
 ',',
 'sequential',
 'arrangement',
 'of',
 'fetal',
 'scans',
 '-',
 'a',
 ':',
 'growth',
 'scan',
 'b',
 ':',
 'triple',
 'marker',
 'c',
 ':',
 'anomalous',
 'scan',
 'and',
 'nt',
 'scan',
 'd',
 ':',
 'all',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]'